In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
y_name = ['ccn_001',
 'ccn_003',
 'ccn_006',
 'CHI',
 'CHI_CCN',
 'D_ALPHA',
 'D_GAMMA',
 'D_ALPHA_CCN',
 'D_GAMMA_CCN',
 'PM25']

In [3]:
x_name = ['TOT_NUM_CONC',
 'TOT_MASS_CONC',
 'pmc_SO4',
 'pmc_NO3',
 'pmc_Cl',
 'pmc_NH4',
 'pmc_ARO1',
 'pmc_ARO2',
 'pmc_ALK1',
 'pmc_OLE1',
 'pmc_API1',
 'pmc_API2',
 'pmc_LIM1',
 'pmc_LIM2',
 'pmc_OC',
 'pmc_BC',
 'pmc_H2O',
 'TEMPERATURE',
 'REL_HUMID',
 'ALT',
 'Z',
 'XLAT',
 'XLONG',
 'h2so4',
 'hno3',
 'hcl',
 'nh3',
 'no',
 'no2',
 'no3',
 'n2o5',
 'hono',
 'hno4',
 'o3',
 'o1d',
 'O3P',
 'oh',
 'ho2',
 'h2o2',
 'co',
 'so2',
 'ch4',
 'c2h6',
 'ch3o2',
 'ethp',
 'hcho',
 'ch3oh',
 'ANOL',
 'ch3ooh',
 'ETHOOH',
 'ald2',
 'hcooh',
 'RCOOH',
 'c2o3',
 'pan',
 'aro1',
 'aro2',
 'alk1',
 'ole1',
 'api1',
 'api2',
 'lim1',
 'lim2',
 'par',
 'AONE',
 'mgly',
 'eth',
 'OLET',
 'OLEI',
 'tol',
 'xyl',
 'cres',
 'to2',
 'cro',
 'open',
 'onit',
 'rooh',
 'ro2',
 'ano2',
 'nap',
 'xo2',
 'xpar',
 'isop',
 'isoprd',
 'isopp',
 'isopn',
 'isopo2',
 'api',
 'lim',
 'dms',
 'msa',
 'dmso',
 'dmso2',
 'ch3so2h',
 'ch3sch2oo',
 'ch3so2',
 'ch3so3',
 'ch3so2oo',
 'ch3so2ch2oo',
 'SULFHOX',
 'P',
 'PB',
 'cos(Time)']

In [4]:
new_time_eval=np.array([-0.5       , -0.46174861, -0.42261826, -0.38268343, -0.34202014,
       -0.3007058 ,  0.70710678,  0.73727734,  0.76604444,  0.79335334,
        0.81915204,  0.84339145])

In [5]:
targ_norm_4d=np.load('targ_norm_4d_eval.npy')

In [17]:
targ_norm_4d.shape

(10, 39, 12, 159, 169)

In [6]:
targ_norm=np.load('targ_norm_flatten_eval.npy')

In [7]:
targ_mean = np.load('targ_mean.npy')
targ_std = np.load('targ_std.npy')


In [8]:
targ_norm_t=np.load('norm_data_timesteps_targ_eval/0targ_norm_eval.npy')

In [9]:
targ_norm_t.shape

(10479690,)

In [15]:
targ_norm_dim=targ_norm_t.reshape(10,39,159,169)

In [21]:
targ_denorm_4d = np.zeros((10, 39, 12, 159, 169)) * np.nan

In [7]:
for it in range(12):
    targ_norm_t=np.load('norm_data_timesteps_targ_eval/'+str(it)+'targ_norm_eval.npy')
    targ_norm_dim=targ_norm_t.reshape(10,39,159,169)
    for iy in range(len(y_name)):
        for iz in range(39):
            y_log=targ_norm_dim[iy,iz,:,:]*targ_std[iy,iz]+targ_mean[iy,iz]
            targ_denorm_4d[iy,iz,it,:,:]=np.exp(y_log)
            


In [10]:
#Next are all features, probably don't need denormalization

In [11]:
#feat_mean=np.load('feat_mean.npy')

In [12]:
#feat_std=np.load('feat_std.npy')

In [13]:
# 102 features + cos(time)
#feat_norm_4d = np.zeros((103, 39, 159, 169)) * np.nan
#feat_norm_t = np.zeros((39*159*169,103)) * np.nan

In [14]:
ft_mean_p=10.31601
ft_std_p=0.9260424
ft_mean_pb=10.305028
ft_std_pb=0.92249596
ft_mean_alt=0.84901726
ft_std_alt=0.8125616
ft_mean_z=8953.694
ft_std_z=6434.8677

for it in range(12):
    print('Time step:'+str(it))
    ds = xr.open_dataset("evaluation.nc").sel(Time=it)
    for ix in range(len(x_name)):
        for iz in range(39):            
            if (ix < len(x_name)-1) and (x_name[ix] != 'Z') and (x_name[ix]!='ALT') and (x_name[ix]!='P') and (x_name[ix]!='PB') and (x_name[ix]!='XLAT') and (x_name[ix]!='XLONG'):
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                x_max = x_new.max()
                x_min = np.where(x_new == 0, x_max, x_new).min()
                x_n = np.where(x_new == 0, x_min, x_new)
                x_log = np.log(x_n)
                feat_norm_4d[ix,iz,:,:]=(x_log - feat_mean[ix,iz])/feat_std[ix,iz]
            if (x_name[ix]=='PB') :
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                x_max = x_new.max()
                x_min = np.where(x_new == 0, x_max, x_new).min()
                x_n = np.where(x_new == 0, x_min, x_new)
                x_log = np.log(x_n)
                feat_norm_4d[ix,iz,:,:]=(x_log - ft_mean_pb)/ft_std_pb
            if (x_name[ix]=='ALT') :
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                x_max = x_new.max()
                x_min = np.where(x_new == 0, x_max, x_new).min()
                x_n = np.where(x_new == 0, x_min, x_new)
                x_log = np.log(x_n)
                feat_norm_4d[ix,iz,:,:]=(x_log - ft_mean_alt)/ft_std_alt
            if (x_name[ix]=='P'):
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                x_add=x_new+x['PB']
                #print(x_add.shape)
                #x_max = x_add.max()
                #x_min = np.where(x_add == 0, x_max, x_add).min()
                #x_n = np.where(x_add == 0, x_min, x_add)
                x_log = np.log(x_add)
                #print(x_log)
                feat_norm_4d[ix,iz,:,:]=(x_log - ft_mean_p)/ft_std_p     
                #print((x_log - ft_mean_p)/ft_std_p)
            if (x_name[ix]=='Z'):
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                feat_norm_4d[ix,iz,:,:]=(x_new - ft_mean_z)/ft_std_z
            if (x_name[ix]=='XLAT') or (x_name[ix]=='XLONG'):
                #print(x_name[ix])
                x = ds.sel(bottom_top=iz)
                x_new=x[x_name[ix]]
                #x_max = x_new.max()
                #x_min = np.where(x_new == 0, x_max, x_new).min()
                #x_n = np.where(x_new == 0, x_min, x_new)
                #x_log = np.log(x_n)
                feat_norm_4d[ix,iz,:,:]=x_new
                
        if ix == 102:
            feat_norm_4d[ix,:,:,:]=new_time_eval[it]
    feat_norm_t=feat_norm_4d[:, :, :, :].flatten().reshape(103,-1).transpose()
    np.save('norm_data_timesteps_feat_eval/'+str(it)+'feat_norm_eval.npy',feat_norm_t)
        


Time step:0
Time step:1
Time step:2
Time step:3
Time step:4
Time step:5
Time step:6
Time step:7
Time step:8
Time step:9
Time step:10
Time step:11
